In [ ]:
import boto3
ec2 = boto3.resource('ec2')
elb = boto3.client('elb')
route53 = boto3.client('route53')

In [ ]:
# add the public ip address to route53 so SSH is easier
response = route53.list_hosted_zones_by_name(
    DNSName='doug-nicholson.net'
)
local_zone_id = response['HostedZones'][0]['Id'][12:]
dns_name='clb.doug-nicholson.net'
response = route53.list_resource_record_sets(
    HostedZoneId=local_zone_id,
    StartRecordName=dns_name,
    StartRecordType='A',
    MaxItems='1'
)
for record_set in response['ResourceRecordSets']:
    if record_set['Name'] == dns_name + '.':
        response = route53.change_resource_record_sets(
            HostedZoneId=local_zone_id,
            ChangeBatch={
                'Changes': [
                    {
                        'Action': 'DELETE',
                        'ResourceRecordSet': record_set
                    }
                ]
            }
        )
        # it seems like waiting will be the right thing to do
        route53_waiter = route53.get_waiter('resource_record_sets_changed')
        route53_waiter.wait(
            Id=response['ChangeInfo']['Id'][8:]
        )

In [ ]:
# Delete the instances
instances_security_group_name = 'instances-security-group'
instance_iterator = ec2.instances.filter(
    Filters=[
        {
            'Name': 'instance.group-name',
            'Values': [
                instances_security_group_name
            ]
        }
    ]
)
for instance in instance_iterator:
    response = instance.terminate()
for instance in instance_iterator:
    instance.wait_until_terminated()

In [ ]:
# Delete the load balancer
load_balancer_name = 'classic-load-balancer'
balancer_security_group_name = 'balancer-security-group'
response = elb.delete_load_balancer(
    LoadBalancerName=load_balancer_name
)

In [ ]:
# Delete the security group, we have to do our own filtering, and
security_group_iterator = ec2.security_groups.filter(
    Filters=[
        {
            'Name': 'group-name',
            'Values': [
                instances_security_group_name,
                balancer_security_group_name
            ]
        }
    ]
)
for security_group in security_group_iterator:
    response = security_group.delete()


In [ ]:
# Success!
print('instances terminated')